In [1]:
import os

In [2]:
pwd

'd:\\CloudDeplyment\\Image-Segmentation-with-AWS\\reserach'

In [3]:
# os.chdir("../")

In [4]:
pwd

'd:\\CloudDeplyment\\Image-Segmentation-with-AWS'

In [28]:
# Entiity

# entitiy
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_dir: Path
    checkpoint_dir: Path

In [29]:
# COnfiguration Manager

from VGGClassifier.constants import *
from VGGClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


In [30]:
class ConfigurationManager:

    def __init__(self, config_filepath= CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):

        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        create_directories([self.config.artifacts_root])



    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:

        config= self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_dir)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_dir)
            ])
        

        prepare_callback_config= PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_dir= Path(config.tensorboard_dir),
            checkpoint_dir= Path(config.checkpoint_dir)
        )

        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training= self.config.training
        prepare_base_model = self.config.prepare_base_model
        params= self.params

        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")
        create_directories([Path(training_data)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config




In [31]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [32]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_dir.as_posix(),
            save_best_only=True
        )



    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [33]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [35]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e    


[2023-08-14 16:54:10,166- INFO]- common - yaml file: config\config.yaml loaded successfully
[2023-08-14 16:54:10,169- INFO]- common - yaml file: params.yaml loaded successfully
[2023-08-14 16:54:10,171- INFO]- common - created directory at: artifacts
[2023-08-14 16:54:10,173- INFO]- common - created directory at: artifacts\prepare_callbacks\checkpoint_dir
[2023-08-14 16:54:10,175- INFO]- common - created directory at: artifacts\prepare_callbacks\tensorboard
[2023-08-14 16:54:10,180- INFO]- common - created directory at: artifacts\data_ingestion\Chicken-fecal-images
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/10
19/19 [==============================] - 20s 1s/step - loss: 13.9284 - accuracy: 0.5574 - val_loss: 6.7753 - val_accuracy: 0.6094
Epoch 2/10
19/19 [==============================] - 27s 1s/step - loss: 11.6499 - accuracy: 0.5439 - val_loss: 0.9691 - val_accuracy: 0.8750
Epoch 3/10
19/19 [==============================] - 28s 1s/step -